In [23]:
from cobra import Reaction, Metabolite
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model

In [24]:
modelc=read_sbml_model('./Models/modelc.xml')

In [25]:
Reaction_0_beta=Reaction('Step0_beta') # Farnesyl diphosphate + isopentenyl diphosphate <=> geranylgeranyl diphosphate + diphosphate
geranylgeranyl_PP=Metabolite(id='Geranylgeranyl_pp_c', compartment='c')
Reaction_0_beta.add_metabolites({modelc.metabolites.frdp_c: -1, # Farnesyl diphosphate
                                 modelc.metabolites.ipdp_c: -1, # Isopentenyl diphosphate
                                 geranylgeranyl_PP: 1, # our newly created metabolites
                                 modelc.metabolites.ppi_c: 1 # diphosphate
                                })

**Reaction 1**

In [26]:
Reaction_1_beta=Reaction('Step1_beta')  # 2 Geranylgeranyl diphosphate <=> Diphosphate + Prephytoene diphosphate
Prephytoene_PP=Metabolite(id='Prephyteoene_pp_c', compartment='c')
Reaction_1_beta.add_metabolites({geranylgeranyl_PP: -2, #Find name of GPP
                                 Prephytoene_PP: 1, # our newly created metabolites
                                 modelc.metabolites.ppi_c: 1 
                                })

**Reaction 2**

In [27]:
Reaction_2_beta=Reaction('Step2_beta')  # Prephytoene diphosphate <=> Diphosphate + 15-cis-Phytoene
Phytoene=Metabolite(id='Phytoene_c', compartment='c')
Reaction_2_beta.add_metabolites({Prephytoene_PP: -1, #made in reaction 1
                              Phytoene: 1, # our newly created metabolite
                              modelc.metabolites.ppi_c: 1 # diphospate
                             })

**Reaction 3**

In [28]:
Reaction_3_beta=Reaction('Step3_beta')  # 15-cis-Phytoene + Acceptor <=> all-trans-Phytofluene + Reduced acceptor
Phytofluene=Metabolite(id='Phytofluene_c', compartment='c')
Reaction_3_beta.add_metabolites({Phytoene: -1, #made in reaction 2
                                 modelc.metabolites.nad_c: -1,
                                 modelc.metabolites.nadh_c: 1,
                                 Phytofluene: 1 # our newly created metabolite
                                })

**Reaction 4**

In [29]:
Reaction_4_beta=Reaction('Step4_beta')  # all-trans-Phytofluene + Acceptor <=> all-trans-zeta-Carotene + Reduced acceptor
zeta_carotene=Metabolite(id='zeta_carotene_c', compartment='c')
Reaction_4_beta.add_metabolites({Phytofluene: -1, #made in reaction 3
                                 modelc.metabolites.nad_c: -1,
                                 modelc.metabolites.nadh_c: 1,
                              zeta_carotene: 1 # our newly created metabolite
                             })

**Reaction 5**

In [30]:
Reaction_5_beta=Reaction('Step5_beta')  # all-trans-zeta-Carotene + Reduced acceptor + Oxygen <=> all-trans-Neurosporene + 2 H2O + Acceptor
neurosporene=Metabolite(id='neurosporene_c', compartment='c')
Reaction_5_beta.add_metabolites({zeta_carotene: -1, #made in reaction 4
                                 modelc.metabolites.o2_c: -1, 
                                 modelc.metabolites.nadh_c: -1,
                                 modelc.metabolites.nad_c: 1,
                                neurosporene: 1, # our newly created metabolite
                                modelc.metabolites.h2o_c: 2 
                             })

**Reaction 6**

In [31]:
Reaction_6_beta=Reaction('Step6_beta')  # all-trans-Neurosporene + Reduced acceptor + Oxygen <=> Lycopene + Acceptor + 2 H2O
Lycopene=Metabolite(id='lycopene_c', compartment='c')
Reaction_6_beta.add_metabolites({neurosporene: -1, #made in reaction 5
                                 modelc.metabolites.o2_c: -1, 
                                 modelc.metabolites.nadh_c: -1,
                                 modelc.metabolites.nad_c: 1,
                              Lycopene: 1, # our newly created metabolite
                                 modelc.metabolites.h2o_c: 2  
                             })

**Reaction 7**

In [32]:
Reaction_7_beta=Reaction('Step7_beta')  # Lycopene <=> gamma-Carotene
Gamma_carotene=Metabolite(id='gamma_carotene_c', compartment='c')
Reaction_7_beta.add_metabolites({Lycopene: -1, #made in reaction 6
                              Gamma_carotene: 1
                             })

**Reaction 8**

In [33]:
Reaction_8_beta=Reaction('Step8_beta')  # gamma-Carotene <=> beta-Carotene
Beta_carotene=Metabolite(id='Beta_carotene_c', compartment='c')
Reaction_8_beta.add_metabolites({Gamma_carotene: -1, #made in reaction 7
                              Beta_carotene: 1
                             })

After defining all of the reaction steps, the reactions are added to the *V. natriegens* GSM:

In [34]:
modelc.add_reactions([Reaction_0_beta])
modelc.add_reactions([Reaction_1_beta])
modelc.add_reactions([Reaction_2_beta])
modelc.add_reactions([Reaction_3_beta])
modelc.add_reactions([Reaction_4_beta])
modelc.add_reactions([Reaction_5_beta])
modelc.add_reactions([Reaction_6_beta])
modelc.add_reactions([Reaction_7_beta])
modelc.add_reactions([Reaction_8_beta])

In order to have flux, bounds have to be implemented for the newly added metabolites:

In [35]:
modelc.add_boundary(modelc.metabolites.Beta_carotene_c, type='demand')

Reaction identifier,DM_Beta_carotene_c
Name,demand
Memory address,0x07f40c1a64978
Stoichiometry,Beta_carotene_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


A *V. natriegens* model containing the beta carotene pathway is saved:

In [36]:
write_sbml_model(modelc,"Models/modelBC.xml")

**Predicting maximal theoretical beta carotene production**

In [37]:
with modelc:
    modelc.objective = modelc.reactions.Step8_beta
    beta_carotene_production = modelc.optimize().objective_value
    print('Max. beta-carotene production [mmol gDW^-1 h^-1]:',beta_carotene_production)

Max. beta-carotene production [mmol gDW^-1 h^-1]: 2.0183670931058955


The maximum production rate of beta carotene in *V. natriegens* with the heterologous beta carotene pathway from *V. campbellii* is 2.02 mmol gDW<sup>-1</sup> h<sup>-1</sup>.

# Extra precursors

**Reaction 1**

In [38]:
Step_1_mva = Reaction('Step_1_mva')  # Acetyl-CoA <=> CoA + Acetoacetyl-CoA
Step_1_mva.add_metabolites({modelc.metabolites.accoa_c: -1, # acetyl-CoA
                            modelc.metabolites.aacoa_c: 1, # acetoacetyl-CoA
                            modelc.metabolites.coa_c: 1 # CoA
                           })

**Reaction 2**

In [39]:
Step_2_mva = Reaction('Step_2_mva')  # (S)-3-Hydroxy-3-methylglutaryl-CoA + CoA <=> Acetyl-CoA + H2O + Acetoacetyl-CoA
Step_2_mva.add_metabolites({modelc.metabolites.accoa_c: -1, # Acetyl-CoA
                            modelc.metabolites.aacoa_c: -1, # Acetoacetyl-CoA
                            modelc.metabolites.h2o_c: -1,
                            modelc.metabolites.hmgcoa_c: 1, # Hydroxymethylglutaryl CoA
                            modelc.metabolites.coa_c: 1
                           })

**Reaction 3**

In [40]:
Step_3_mva = Reaction('Step_3_mva')  # (R)-Mevalonate + CoA + 2 NAD+ <=> (S)-3-Hydroxy-3-methylglutaryl-CoA + 2 NADH + 2 H+
mevalonate = Metabolite(id='Mevalonate_c', compartment='c')
Step_3_mva.add_metabolites({modelc.metabolites.hmgcoa_c: -1,
                            modelc.metabolites.nadh_c: -2,
                            modelc.metabolites.h_c: -2,
                            mevalonate: 1,
                            modelc.metabolites.coa_c: 1,
                            modelc.metabolites.nad_c: 2
                           })

**Reaction 4**

In [41]:
Step_4_mva = Reaction('Step_4_mva')  # ATP + (R)-Mevalonate <=> ADP + (R)-5-Phosphomevalonate
mevalonate_5P = Metabolite(id='Phosphomevalonate_c', compartment='c')
Step_4_mva.add_metabolites({mevalonate: -1,
                            modelc.metabolites.atp_c: -1,
                            mevalonate_5P: 1,
                            modelc.metabolites.adp_c: 1
                           })

**Reaction 5**

In [42]:
Step_5_mva = Reaction('Step_5_mva')  # ATP + (R)-5-Phosphomevalonate <=> ADP + (R)-5-Diphosphomevalonate
mevalonate_5PP = Metabolite(id='Diphosphomevalonate_c', compartment='c')
Step_5_mva.add_metabolites({mevalonate_5P: -1,
                            modelc.metabolites.atp_c: -1,
                            mevalonate_5PP: 1,
                            modelc.metabolites.adp_c: 1
                           })

**Reaction 6**

In [43]:
Step_6_mva = Reaction('Step_6_mva')  # ATP + (R)-5-Diphosphomevalonate <=> ADP + Orthophosphate + Isopentenyl diphosphate + CO2
ortophosphate = Metabolite(id='Ortophosphate_c', compartment='c')
Step_6_mva.add_metabolites({mevalonate_5PP: -1,
                            modelc.metabolites.atp_c: -1,
                            modelc.metabolites.ipdp_c: 1,
                            modelc.metabolites.adp_c: 1,
                            ortophosphate: 1,
                            modelc.metabolites.co2_c: 1
                           })

**Adding reactions to model**

In [44]:
with modelc:
    modelc.add_reactions([Step_1_mva])
    modelc.add_reactions([Step_2_mva])
    modelc.add_reactions([Step_3_mva])
    modelc.add_reactions([Step_4_mva])
    modelc.add_reactions([Step_5_mva])
    modelc.add_reactions([Step_6_mva])
    write_sbml_model(modelc,"Models/modelBCMVA.xml")
    
    
    modelc.objective = modelc.reactions.Step8_beta
    beta_carotene_production = modelc.optimize().objective_value
    print('Max. beta-carotene production [mmol gDW^-1 h^-1]:', beta_carotene_production)


Max. beta-carotene production [mmol gDW^-1 h^-1]: 2.0183670931058977


The maximum production rate of beta carotene in *V. natriegens* with the heterologous beta carotene pathway from *V. campbellii* and the heterologous MVA pathway from *L. acidophilus* is 2.02 mmol gDW<sup>-1</sup> h<sup>-1</sup>. So there is no difference in the beta carotene production with the MVA pathway introduced.